In [1]:
#Title 

In [2]:
#Deliverable 1 Population Statistics

In [2]:
!pip install wbdata

  Using cached wbdata-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached dateparser-1.2.1-py3-none-any.whl.metadata (29 kB)
  Using cached shelved_cache-0.3.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached tabulate-0.8.10-py3-none-any.whl.metadata (25 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzlocal-5.3-py3-none-any.whl.metadata (7.6 kB)
Using cached wbdata-1.0.0-py3-none-any.whl (18 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached dateparser-1.2.1-py3-none-any.whl (295 kB)
Using cached shelved_cache-0.3.1-py3-none-any.whl (7.8 kB)
Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzlocal-5.3-py3-none-any.whl (17 kB)
  Attempting uninstall: pytz
    Found exist

In [3]:
import wbdata
wbdata.get_topics()

Key '4680761537058067707' not in persistent cache.
Key '-6556621766149829814' not in persistent cache.
Key '1796291256731634009' not in persistent cache.
Key '-2347828713703408156' not in persistent cache.
Key '7247804645467710899' not in persistent cache.
Key '6283088729352311478' not in persistent cache.
Key '-4500465859374313704' not in persistent cache.
Key '-2463677044534739414' not in persistent cache.
Key '5435407263865452070' not in persistent cache.
Key '-5348147324861173112' not in persistent cache.
Key '322383150133450411' not in persistent cache.
Key '7720052053025674735' not in persistent cache.
Key '3938854866389376216' not in persistent cache.
Key '248563610435606390' not in persistent cache.
Key '7457924380937260485' not in persistent cache.
Key '8879481053491326024' not in persistent cache.
Key '2730178671166334020' not in persistent cache.
Key '-6048193505670341323' not in persistent cache.
Key '8031799588004466616' not in persistent cache.
Key '-3205583715203430905' 

  id  value
----  -------------------------------
   1  Agriculture & Rural Development
   2  Aid Effectiveness
   3  Economy & Growth
   4  Education
   5  Energy & Mining
   6  Environment
   7  Financial Sector
   8  Health
   9  Infrastructure
  10  Social Protection & Labor
  11  Poverty
  12  Private Sector
  13  Public Sector
  14  Science & Technology
  15  Social Development
  16  Urban Development
  17  Gender
  18  Millenium development goals
  19  Climate Change
  20  External Debt
  21  Trade

In [4]:
import pandas as pd
import numpy as np

In [5]:
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Glob

In [6]:
indicators = wbdata.get_indicators(source=40)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

In [7]:
sexdict = {'Male':'MA', 'Female':'FE'}
def population(year,sex,age_range,place):
    lower, upper = age_range[0], age_range[1]
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df = df[df['date'] == f'{year}-01-01'] # selecting relevant year
        df.drop('date', axis=1, inplace=True)
        extra = ((yrs[-1][1] - upper) / 5) * (df.iloc[:,-1].to_list()[0]) # controls for age ranges not on bounds, could change to 
                                                                          # year by year subtraction)
    return df.sum(axis=1) - extra

In [8]:
population(1990, 'Male', (10,33), 'CHN')

33    263801536.6
dtype: float64

In [9]:
def population_data_frames(year, sex, age_range, place):
    lower, upper = age_range[0], age_range[1]
    assert upper >= lower, "Invalid date range!" #Part of Deliverable 2
    assert sex in ["Male", "Female", "Both"], "Invalid sex Please use one of the following: Male, Female, Both"
    if sex == "Both":
        return population_data_frames(year,"Male",age_range,place).merge(population_data_frames(year,"Female",age_range,place), on="date")
    if upper > 80:
        upper = 'UP'
    if upper == lower + 4:
        label = f'SP.POP.{lower:02d}{upper}.{sexdict[sex]}.IN'
        df = wbdata.get_dataframe(label,country=place,parse_dates=True)
    else:
        yrs = [(x, x+4) for x in range(lower, upper, 5)]
        ranges = [f'{x[0]:02d}{x[1]:02d}' for x in yrs]
        var = [f"SP.POP.{x}.{sexdict[sex]}" for x in ranges]
        feed = {x: f"{sex} {ranges[var.index(x)]}" for x in var} # constructs variable dictionary to be fed to wbdata
        
        df = wbdata.get_dataframe(feed,country=place,parse_dates=True).reset_index()
        df['country'] = place
        df.set_index(['country', 'date'], inplace=True)
    return df

In [10]:
population_data_frames(1990, 'Male', (0,80), 'CHN')

Male 0004   Male 0509   Male 1014   Male 1519  \
country date                                                         
CHN     2023-01-01  30390275.0  46921541.0  47697475.0  43619845.0   
        2022-01-01  34407301.0  47977432.0  46945694.0  42880142.0   
        2021-01-01  38802141.0  48127269.0  46077575.0  42441714.0   
        2020-01-01  42401148.0  48099312.0  45390757.0  42444894.0   
        2019-01-01  45006875.0  48004396.0  44534610.0  42626158.0   
...                        ...         ...         ...         ...   
        1964-01-01  53250448.0  48925593.0  45210547.0  32772310.0   
        1963-01-01  47951039.0  50749733.0  43265071.0  31245247.0   
        1962-01-01  44777918.0  50854256.0  40877563.0  29902774.0   
        1961-01-01  46459365.0  50481643.0  38596810.0  29125584.0   
        1960-01-01  51416955.0  50098089.0  36823111.0  29027168.0   

                     Male 2024   Male 2529   Male 3034   Male 3539  \
country date                                                         
CHN     2023-01-01  42690196.0  45996968.0  58571312.0  58164459.0   
        2022-01-01  43079433.0  47322938.0  61347262.0  56040230.0   
        2021-01-01  43637348.0  48943931.0  63431979.0  54207913.0   
        2020-01-01  44208685.0  51917714.0  63016755.0  52572479.0   
        2019-01-01  45014490.0  55706672.0  60783908.0  51377332.0   
...                        ...         ...         ...         ...   
        1964-01-01  27179605.0  26566937.0  24330081.0  21031169.0   
        1963-01-01  27269359.0  26878889.0  23871048.0  20785092.0   
        1962-01-01  27482213.0  26824171.0  23384228.0  20378091.0   
        1961-01-01  27700640.0  26562875.0  23028989.0  20175956.0   
        1960-01-01  28092415.0  26459425.0  22874774.0  20178976.0   

                     Male 4044   Male 4549   Male 5054   Male 5559  \
country date                                                         
CHN     2023-01-01  50128672.0  49849324.0  61288205.0  56295989.0   
        2022-01-01  48623953.0  52814513.0  61302770.0  56011299.0   
        2021-01-01  47300851.0  55783379.0  60422799.0  53103266.0   
        2020-01-01  47188710.0  58708249.0  59481364.0  47982121.0   
        2019-01-01  48144457.0  61076492.0  58472576.0  42984534.0   
...                        ...         ...         ...         ...   
        1964-01-01  18185693.0  15660355.0  13340206.0  10840068.0   
        1963-01-01  17811841.0  15758791.0  13103423.0  10794563.0   
        1962-01-01  17531795.0  15710166.0  12919455.0  10776768.0   
        1961-01-01  17398685.0  15571913.0  12923650.0  10792560.0   
        1960-01-01  17445180.0  15573208.0  13126586.0  10884917.0   

                     Male 6064   Male 6569   Male 7074   Male 7579  
country date                                                        
CHN     2023-01-01  36317207.0  36029698.0  26676047.0  15052265.0  
        2022-01-01  33543614.0  35762285.0  24926689.0  14205563.0  
        2021-01-01  34097647.0  35022489.0  23177262.0  13510140.0  
        2020-01-01  36274094.0  33979409.0  21574911.0  13015232.0  
        2019-01-01  38374180.0  32428084.0  20069867.0  12627031.0  
...                        ...         ...         ...         ...  
        1964-01-01   8229597.0   5325280.0   3127717.0   1527155.0  
        1963-01-01   8016810.0   5346612.0   3103485.0   1514318.0  
        1962-01-01   7829214.0   5339866.0   3082473.0   1507571.0  
        1961-01-01   7747646.0   5362310.0   3131356.0   1537572.0  
        1960-01-01   7858547.0   5468648.0   3289905.0   1628784.0  

[64 rows x 16 columns]

In [11]:
population_data_frames(1990, 'Female', (10,33), 'CHN')

Female 1014  Female 1519  Female 2024  Female 2529  \
country date                                                             
CHN     2023-01-01   40892243.0   37182969.0   36605841.0   40184117.0   
        2022-01-01   40161573.0   36559731.0   37035577.0   41562725.0   
        2021-01-01   39352043.0   36210584.0   37629021.0   43238213.0   
        2020-01-01   38718791.0   36255224.0   38255106.0   46168527.0   
        2019-01-01   37959894.0   36469477.0   39112543.0   49873080.0   
...                         ...          ...          ...          ...   
        1964-01-01   43236900.0   30062519.0   24208269.0   23499927.0   
        1963-01-01   41174542.0   28433763.0   24118305.0   23923846.0   
        1962-01-01   38550727.0   27084802.0   24120873.0   24034501.0   
        1961-01-01   35990952.0   26262486.0   24208928.0   23901128.0   
        1960-01-01   33964856.0   25955016.0   24590972.0   23794913.0   

                    Female 3034  
country date                     
CHN     2023-01-01   52861422.0  
        2022-01-01   55702933.0  
        2021-01-01   57929324.0  
        2020-01-01   57874577.0  
        2019-01-01   56135406.0  
...                         ...  
        1964-01-01   22007969.0  
        1963-01-01   21660173.0  
        1962-01-01   21287773.0  
        1961-01-01   21019399.0  
        1960-01-01   20951533.0  

[64 rows x 5 columns]

In [12]:
population_data_frames(1990, 'Both', (0,80), 'CHN')

,Male 0004,Male 0509,Male 1014,Male 1519,Male 2024,Male 2529,Male 3034,Male 3539,Male 4044,Male 4549,...,Female 3034,Female 3539,Female 4044,Female 4549,Female 5054,Female 5559,Female 6064,Female 6569,Female 7074,Female 7579
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,30390275.0,46921541.0,47697475.0,43619845.0,42690196.0,45996968.0,58571312.0,58164459.0,50128672.0,49849324.0,...,52861422.0,54111081.0,47658004.0,48165337.0,60432093.0,56825905.0,37771419.0,39685195.0,31522585.0,18369107.0
2022-01-01,34407301.0,47977432.0,46945694.0,42880142.0,43079433.0,47322938.0,61347262.0,56040230.0,48623953.0,52814513.0,...,55702933.0,52391805.0,46321543.0,51130200.0,60536192.0,56551300.0,35002089.0,39507542.0,29106909.0,17241445.0
2021-01-01,38802141.0,48127269.0,46077575.0,42441714.0,43637348.0,48943931.0,63431979.0,54207913.0,47300851.0,55783379.0,...,57929324.0,50904100.0,45150868.0,54098374.0,59738602.0,53597653.0,35660756.0,38793955.0,26676375.0,16375324.0
2020-01-01,42401148.0,48099312.0,45390757.0,42444894.0,44208685.0,51917714.0,63016755.0,52572479.0,47188710.0,58708249.0,...,57874577.0,49544962.0,45127659.0,57028683.0,58862173.0,48420766.0,37969600.0,37695737.0,24481027.0,15706266.0
2019-01-01,45006875.0,48004396.0,44534610.0,42626158.0,45014490.0,55706672.0,60783908.0,51377332.0,48144457.0,61076492.0,...,56135406.0,48557018.0,46127571.0,59426420.0,57903342.0,43397958.0,40213974.0,35946934.0,22477861.0,15164232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964-01-01,53250448.0,48925593.0,45210547.0,32772310.0,27179605.0,26566937.0,24330081.0,21031169.0,18185693.0,15660355.0,...,22007969.0,19483299.0,17392150.0,15431263.0,13393101.0,11786993.0,9466826.0,6632284.0,4460153.0,2517810.0
1963-01-01,47951039.0,50749733.0,43265071.0,31245247.0,27269359.0,26878889.0,23871048.0,20785092.0,17811841.0,15758791.0,...,21660173.0,19354179.0,17095469.0,15479965.0,13256152.0,11717468.0,9144942.0,6641740.0,4377684.0,2488273.0
1962-01-01,44777918.0,50854256.0,40877563.0,29902774.0,27482213.0,26824171.0,23384228.0,20378091.0,17531795.0,15710166.0,...,21287773.0,19076777.0,16884402.0,15370624.0,13165906.0,11656254.0,8846123.0,6626940.0,4305783.0,2473695.0


In [13]:
#assert population(year=2000,sex='Male',age_range=(0,100),place='WLD') > 0.0000000000000000001,"Too few males!"
population_data_frames(year=1990,sex='Male',age_range=(0,80),place='WLD')

Male 0004    Male 0509    Male 1014    Male 1519  \
country date                                                             
WLD     2023-01-01  334765288.0  353677898.0  350908323.0  330734289.0   
        2022-01-01  339582664.0  355425122.0  347895623.0  327019295.0   
        2021-01-01  345189042.0  355703002.0  344122479.0  324017775.0   
        2020-01-01  350515471.0  355003268.0  340399749.0  322082499.0   
        2019-01-01  354290946.0  353558134.0  336199668.0  320243605.0   
...                         ...          ...          ...          ...   
        1964-01-01  238500833.0  207779867.0  183539354.0  148168760.0   
        1963-01-01  230073603.0  205823961.0  177924352.0  142070274.0   
        1962-01-01  223504268.0  202091753.0  171734513.0  136888120.0   
        1961-01-01  221655154.0  197980983.0  165710593.0  132998073.0   
        1960-01-01  223060092.0  193928438.0  159370701.0  131387366.0   

                      Male 2024    Male 2529    Male 3034    Male 3539  \
country date                                                             
WLD     2023-01-01  315810689.0  307639362.0  310933348.0  295368359.0   
        2022-01-01  314174911.0  307658633.0  311976794.0  290144561.0   
        2021-01-01  312601224.0  307806041.0  311921453.0  284849609.0   
        2020-01-01  311226388.0  309641234.0  309357029.0  279781371.0   
        2019-01-01  310119231.0  312139575.0  304364288.0  274463093.0   
...                         ...          ...          ...          ...   
        1964-01-01  127111370.0  121082975.0  110092224.0   99753275.0   
        1963-01-01  126982400.0  119446331.0  109213285.0   97418666.0   
        1962-01-01  127032057.0  117484451.0  108231564.0   94798902.0   
        1961-01-01  126755054.0  115629420.0  106926150.0   92445551.0   
        1960-01-01  126062243.0  114504362.0  105334423.0   90480803.0   

                      Male 4044    Male 4549    Male 5054    Male 5559  \
country date                                                             
WLD     2023-01-01  264576185.0  237668148.0  229011272.0  203254223.0   
        2022-01-01  258341893.0  237254249.0  226237535.0  201022206.0   
        2021-01-01  252519605.0  237465269.0  222660554.0  196552263.0   
        2020-01-01  248255758.0  238154909.0  219307981.0  189959120.0   
        2019-01-01  244893214.0  237904370.0  215885465.0  182647014.0   
...                         ...          ...          ...          ...   
        1964-01-01   84206017.0   69811592.0   65484175.0   55288443.0   
        1963-01-01   81579394.0   69993009.0   64683739.0   54295780.0   
        1962-01-01   78986036.0   70462994.0   63816390.0   53296751.0   
        1961-01-01   76693913.0   70871501.0   63076098.0   52308504.0   
        1960-01-01   75021984.0   71134393.0   62418640.0   51506016.0   

                      Male 6064    Male 6569   Male 7074   Male 7579  
country date                                                          
WLD     2023-01-01  162420898.0  135500657.0  99573175.0  61764493.0  
        2022-01-01  156823861.0  132342870.0  95533321.0  58615814.0  
        2021-01-01  154831165.0  129139177.0  91827555.0  56214775.0  
        2020-01-01  154564673.0  125884632.0  87855497.0  55019059.0  
        2019-01-01  153668727.0  121764954.0  83124933.0  54115599.0  
...                         ...          ...         ...         ...  
        1964-01-01   43363520.0   30525542.0  19786191.0  11743222.0  
        1963-01-01   42219932.0   29780479.0  19465748.0  11437987.0  
        1962-01-01   41096068.0   29043510.0  19225580.0  11104459.0  
        1961-01-01   40091553.0   28350434.0  19034652.0  10830633.0  
        1960-01-01   39191224.0   27805941.0  18929537.0  10637970.0  

[64 rows x 16 columns]

Visualisations

In [14]:
import wbdata
import pandas as pd
import plotly.express as px

In [40]:
#Mortality Rate per 1,000 Adults 

variable_labels = {"SP.DYN.AMRT.FE": "Adult female",
                   "SP.DYN.AMRT.MA": "Adult male"}

China_pop = wbdata.get_dataframe(variable_labels, country = "CHN") 

China_pop.index = pd.to_datetime(China_pop.index)
China_pop.index = China_pop.index.year
China_pop = China_pop.reset_index().melt(id_vars = "date", var_name = "Category", value_name = "Mortality Rate")

fig = px.line(China_pop,x = "date", y = "Mortality Rate", color = "Category", title = "Mortality Rate per 1,000 Adults (1960-2021)",labels = {"date": "Year", "Mortality Rate": "Mortality Rate Per 1,000"})
fig.show()

In [38]:
#GDP of China (1960-2021)

indicator = {"NY.GDP.MKTP.CD": "GDP (current US$)"}
gdp_data = wbdata.get_dataframe(indicator, country = "CHN", date = (pd.to_datetime('1960-01-01'), pd.to_datetime('2023-12-31')))
gdp_data.index = pd.to_datetime(gdp_data.index)
gdp_data = gdp_data.reset_index()

fig = px.line(gdp_data, x = "date", y = "GDP (current US$)", title = "GDP of China (1960-2021)", labels = {"date": "Year", "GDP (current US$)": "GDP (current US$)"},)
fig.show()

In [36]:
#Urban Population as a Percentage of Total Population (1960-2021)

variable_labels = {
    "SP.URB.TOTL.IN.ZS": "Urban Population (% of total)"
}

China_urban_pop = wbdata.get_dataframe(variable_labels, country="CHN")

China_urban_pop.index = pd.to_datetime(China_urban_pop.index)
China_urban_pop.index = China_urban_pop.index.year

China_urban_pop = China_urban_pop.reset_index()
China_urban_pop.columns = ['Year', 'Urban Population (%)']

fig = px.line(China_urban_pop, x = "Year", y = "Urban Population (%)", title = "Urban Population as Percentage of Total Population: 1960-2021", labels = {"Year": "Year", "Urban Population (%)": "Urban Population (%)"})
fig.show()

In [37]:
#China's Total Population vs GDP (1960-2021)

import wbdata
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

indicator_gdp = {"NY.GDP.MKTP.CD": "GDP (current US$)"}
indicator_pop = {"SP.POP.TOTL": "Total Population"}

gdp_data = wbdata.get_dataframe(indicator_gdp, country = "CHN", date = (pd.to_datetime('1960-01-01'), pd.to_datetime('2023-12-31')))
population_data = wbdata.get_dataframe(indicator_pop, country="CHN", date = (pd.to_datetime('1960-01-01'), pd.to_datetime('2023-12-31')))

merged_data = pd.merge(gdp_data, population_data, left_index=True, right_index = True, how = 'inner')
merged_data = merged_data.reset_index()

fig = make_subplots(specs = [[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x = merged_data["date"],
        y = merged_data["GDP (current US$)"],
        mode = "lines",
        name = "GDP (current US$)",
    ),
    secondary_y = False,
)

fig.add_trace(
    go.Scatter(
        x = merged_data["date"],
        y = merged_data["Total Population"],
        mode = "lines",
        name = "Total Population",
        line = dict(color = "red"),  
    ),
    secondary_y=True,
)

fig.update_layout(
    title = "GDP and Population Trend for China (1960-2021)",
    xaxis_title = "Year",
    plot_bgcolor = "white",  
    paper_bgcolor = "white",  
)

fig.update_xaxes(autorange="reversed")
fig.update_yaxes(title_text="GDP (current US$)", secondary_y = False)
fig.update_yaxes(title_text="Total Population", secondary_y = True)

fig.show()

Total Population: China continues to see a growth in population. However, the rate at which the population is increasing at has decreased since the One child Policy was implemented in 1979 (shallower slope). As the policy shifted to a two child policy in 2016, the growth rate remains lower than it did prior to implmenetation of the One Child Policy. These impacts will continue to impact China's economy there will be much less young people of working age to support the older population and contribute to the rapid increase in GDP. 

GDP: China as seen a slow, but consistent increase in GDP in the 1900s until the early 2000s where it saw a rapid rise.

Despite the decrease in growth rate in population over time, its GDP has seen incredible growth. 

In [44]:
#China Fertility Rate Over Time (1960-2021)
import wbdata
import pandas as pd
import plotly.graph_objects as go

!pip install cufflinks
import cufflinks as cf
cf.go_offline()

tfr_indicator = {'SP.DYN.TFRT.IN': 'Fertility Rate'}

country = 'CHN'  # China
start_year = 1960
end_year = 2022

fertility_data = wbdata.get_dataframe(tfr_indicator, country = country)
fertility_data.reset_index(inplace = True)
fertility_data['date'] = fertility_data['date'].astype(int)

fig = go.Figure()
fig.add_trace(go.Scatter(x = fertility_data['date'], y = fertility_data['Fertility Rate'], mode = 'lines+markers', name = 'China Fertility Rate'))

fig.update_layout(
    title = 'China Fertility Rate Over Time (1960-2022)',
    xaxis_title = 'Year',
    yaxis_title = 'Fertility Rate (Births per Woman)',
    showlegend = True,
    xaxis = dict(tickmode='linear', tickangle = 90, tickfont = dict(size = 12))
)

fig.show()

In [45]:
population_indicators = {
    'SP.URB.TOTL': 'Urban Population',
    'SP.RUR.TOTL': 'Rural Population'
}

country = 'CHN'  # China
start_year = 1960
end_year = 2023

population_data = wbdata.get_dataframe(population_indicators, country=country)
population_data.reset_index(inplace = True)
population_data['date'] = population_data['date'].astype(int)

fig = go.Figure()
fig.add_trace(go.Scatter(x = population_data['date'], y = population_data['Urban Population'], mode = 'lines+markers', name = 'Urban Population'))
fig.add_trace(go.Scatter(x = population_data['date'], y = population_data['Rural Population'], mode = 'lines+markers', name = 'Rural Population'))

fig.update_layout(
    title = 'China Urban vs Rural Population Growth (1960-2023)',
    xaxis_title = 'Year',
    yaxis_title = 'Population',
    showlegend = True,
    xaxis = dict(tickmode = 'linear', tickangle = 90, tickfont = dict(size = 12))
)

fig.show()

In [47]:
#China Population Growth Rate (1960 -2021)
population_growth_indicator = {'SP.POP.GROW': 'Population Growth Rate'}

country = 'CHN'  # China
start_year = 1960
end_year = 2023

population_growth_data = wbdata.get_dataframe(population_growth_indicator, country=country)
population_growth_data.reset_index(inplace=True)
population_growth_data['date'] = population_growth_data['date'].astype(int)

fig = go.Figure()
fig.add_trace(go.Scatter(x=population_growth_data['date'], y=population_growth_data['Population Growth Rate'], mode='lines+markers', name='Population Growth Rate'))
fig.update_layout(
    title='China Population Growth Rate (1960-2023)',
    xaxis_title='Year',
    yaxis_title='Growth Rate (%)',
    showlegend=True,
    xaxis=dict(tickmode='linear', tickangle=90, tickfont=dict(size=12))
)

fig.show()